# Project 3 

#### Jason Kim 
#### Braulio Lopez

### Part 1 

#### Importing the libraries

In [1]:
pip install tensorflow_datasets --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 49.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 230.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 252.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 256.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 260.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 227.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 306.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 298.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 313.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 299.1 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=435b66dea9abfafcaeaf67d6e9a677a417abb832521649be4c7105e70d

In [2]:
import pandas as pd
import seaborn as sns
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from pathlib import Path
import keras
from keras.datasets import mnist
import random
import shutil
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

/tmp/ipykernel_485/952184717.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-11 09:34:14.712619: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 09:34:14.759104: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 09:34:14.759148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cu

#### Loading Dataset 

In [3]:
try:
    shutil.rmtree("data_all_modified-cnn-split/train")
    shutil.rmtree("data_all_modified-cnn-split/test")
except:
    pass

In [4]:
Path("data_all_modified-cnn-split/train/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-cnn-split/train/no_damage").mkdir(parents=True, exist_ok=True)

Path("data_all_modified-cnn-split/test/damage").mkdir(parents=True, exist_ok=True)
Path("data_all_modified-cnn-split/test/no_damage").mkdir(parents=True, exist_ok=True)

In [5]:
all_damage_file_paths = os.listdir('data_all_modified/damage')
all_no_damage_file_paths = os.listdir('data_all_modified/no_damage')

In [6]:
train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train no damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test no damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))

train damage image count:  11336
test damage image count:  2834
len of overlap:  0
train no damage image count:  5721
test no damage image count:  1431
len of overlap:  0


In [7]:
# ensure to copy the images to the directories
for p in train_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-cnn-split/train/damage', p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/damage', p), os.path.join('data_all_modified-cnn-split/test/damage', p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-cnn-split/train/no_damage', p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('data_all_modified/no_damage', p), os.path.join('data_all_modified-cnn-split/test/no_damage', p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("data_all_modified-cnn-split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("data_all_modified-cnn-split/train/no_damage")))
print("Files in test/damage: ", len(os.listdir("data_all_modified-cnn-split/test/damage")))
print("Files in test/no_damage: ", len(os.listdir("data_all_modified-cnn-split/test/no_damage")))

Files in train/damage:  11336
Files in train/no_damage:  5721
Files in test/damage:  2834
Files in test/no_damage:  1431


#### Data Processing

In [8]:
from PIL import Image
import os

# Path to the directory containing your images
dataset_dir = 'data_all_modified/no_damage'

image_dimensions = {}

# Loop through each image file in the directory
for filename in os.listdir(dataset_dir):
    if filename.endswith('.jpeg') or filename.endswith('.png'):  # Adjust file extensions as needed
        # Open the image using PIL
        img = Image.open(os.path.join(dataset_dir, filename))
        # Get the dimensions of the image
        width, height = img.size
        # Add dimensions to the dictionary
        if (width, height) in image_dimensions:
            image_dimensions[(width, height)] += 1
        else:
            image_dimensions[(width, height)] = 1

# Print summary of image dimensions
for dimensions, count in image_dimensions.items():
    print(f"{count} images with dimensions {dimensions[0]}x{dimensions[1]}")

7152 images with dimensions 128x128


In [9]:
train_data_dir = 'data_all_modified-cnn-split/train'
batch_size = 32
# target image size
img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [10]:
test_data_dir = 'data_all_modified-cnn-split/test/'

batch_size = 2

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 4265 files belonging to 2 classes.


### Part 2

#### ANN

In [11]:
from tensorflow.keras import models, layers

model_ann = models.Sequential()

# Adjust the input_shape to match the actual image size
model_ann.add(layers.Flatten(input_shape=(128, 128, 3)))  # This line is changed

model_ann.add(layers.Dense(512, activation='relu'))
model_ann.add(layers.Dense(256, activation='relu'))

# Add additional hidden layers
model_ann.add(layers.Dense(128, activation='relu'))
model_ann.add(layers.Dense(64, activation='relu'))

model_ann.add(layers.Dense(1, activation='sigmoid'))  # For binary classification

model_ann.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

model_ann.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 49152)             0         
                                                                 
 dense (Dense)               (None, 512)               25166336  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 25338881 (96.66 MB)
Trainable params: 2533

In [12]:
history_ann = model_ann.fit(
    train_rescale_ds,
    epochs=20,
    validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 58s 133ms/step - loss: 0.9443 - accuracy: 0.6463 - val_loss: 0.8573 - val_accuracy: 0.4057
Epoch 2/20
427/427 [==============================] - 63s 147ms/step - loss: 0.6070 - accuracy: 0.6843 - val_loss: 0.5802 - val_accuracy: 0.6869
Epoch 3/20
427/427 [==============================] - 66s 155ms/step - loss: 0.5930 - accuracy: 0.6966 - val_loss: 0.5729 - val_accuracy: 0.6740
Epoch 4/20
427/427 [==============================] - 64s 150ms/step - loss: 0.6272 - accuracy: 0.6612 - val_loss: 0.6350 - val_accuracy: 0.6681
Epoch 5/20
427/427 [==============================] - 65s 152ms/step - loss: 0.6376 - accuracy: 0.6637 - val_loss: 0.6363 - val_accuracy: 0.6681
Epoch 6/20
427/427 [==============================] - 63s 148ms/step - loss: 0.6389 - accuracy: 0.6637 - val_loss: 0.6358 - val_accuracy: 0.6681
Epoch 7/20
427/427 [==============================] - 68s 159ms/step - loss: 0.6391 - accuracy: 0.6637 - val_loss: 0.6359 - val_ac

In [13]:
test_loss, test_accuracy = model_ann.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.6381585597991943
Test Accuracy: 0.6644783020019531


In [14]:
'''
from pathlib import Path
from tensorflow.keras.models import load_model

# Define the path where you want to save the model
model_path = Path('./models/ANN_Model.h5')

# Check if the file already exists
if model_path.exists():
    # If the file exists, remove it
    model_path.unlink()

# Save the model
model_ann.save(model_path)
'''

"\nfrom pathlib import Path\nfrom tensorflow.keras.models import load_model\n\n# Define the path where you want to save the model\nmodel_path = Path('./models/ANN_Model.h5')\n\n# Check if the file already exists\nif model_path.exists():\n    # If the file exists, remove it\n    model_path.unlink()\n\n# Save the model\nmodel_ann.save(model_path)\n"

#### LeNet-5

In [17]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(128,128,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons
model_lenet5.add(layers.Dense(1, activation='sigmoid'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d_2 (Avera  (None, 63, 63, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_3 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_2 (Flatten)         (None, 14400)             0         
                                                                 
 dense_8 (Dense)             (None, 120)              

In [18]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 25s 57ms/step - loss: 0.5697 - accuracy: 0.7219 - val_loss: 0.4680 - val_accuracy: 0.7960
Epoch 2/20
427/427 [==============================] - 24s 56ms/step - loss: 0.4533 - accuracy: 0.8088 - val_loss: 0.4200 - val_accuracy: 0.8118
Epoch 3/20
427/427 [==============================] - 24s 56ms/step - loss: 0.4109 - accuracy: 0.8323 - val_loss: 0.4264 - val_accuracy: 0.8434
Epoch 4/20
427/427 [==============================] - 25s 57ms/step - loss: 0.3901 - accuracy: 0.8481 - val_loss: 0.5452 - val_accuracy: 0.7593
Epoch 5/20
427/427 [==============================] - 25s 58ms/step - loss: 0.3709 - accuracy: 0.8605 - val_loss: 0.4349 - val_accuracy: 0.8317
Epoch 6/20
427/427 [==============================] - 25s 58ms/step - loss: 0.3476 - accuracy: 0.8719 - val_loss: 0.3566 - val_accuracy: 0.8563
Epoch 7/20
427/427 [==============================] - 25s 58ms/step - loss: 0.3219 - accuracy: 0.8811 - val_loss: 0.3176 - val_accuracy:

In [19]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.322668194770813
Test Accuracy: 0.8590855598449707


In [ ]:
'''
from pathlib import Path
from tensorflow.keras.models import load_model

# Define the path where you want to save the model
model_path = Path('./models/Lenet5_cnn.h5')

# Check if the file already exists
if model_path.exists():
    # If the file exists, remove it
    model_path.unlink()

# Save the model
model_lenet5.save(model_path)
'''

#### Alternate-Lenet-5 CNN architecture

In [20]:
from keras.layers import LeakyReLU
from keras.regularizers import l2
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

model = models.Sequential()

model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (128,128,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128,(3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(layers.Dropout(0.5))

model.add(layers.Dense(512, activation = 'relu', kernel_regularizer = l2(1e-4)))
model.add(layers.Dense(1, activation = 'sigmoid'))

#compile the model with RMSprob with learning rate
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(learning_rate=1e-4), metrics = ['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)      

In [21]:
history = model.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
427/427 [==============================] - 94s 216ms/step - loss: 0.6522 - accuracy: 0.7271 - val_loss: 0.5141 - val_accuracy: 0.8006
Epoch 2/20
427/427 [==============================] - 88s 205ms/step - loss: 0.4984 - accuracy: 0.8232 - val_loss: 0.4483 - val_accuracy: 0.8631
Epoch 3/20
427/427 [==============================] - 91s 213ms/step - loss: 0.4226 - accuracy: 0.8608 - val_loss: 0.3538 - val_accuracy: 0.8986
Epoch 4/20
427/427 [==============================] - 90s 210ms/step - loss: 0.3506 - accuracy: 0.8925 - val_loss: 0.4038 - val_accuracy: 0.8666
Epoch 5/20
427/427 [==============================] - 92s 214ms/step - loss: 0.2895 - accuracy: 0.9147 - val_loss: 0.3177 - val_accuracy: 0.9088
Epoch 6/20
427/427 [==============================] - 92s 216ms/step - loss: 0.2494 - accuracy: 0.9299 - val_loss: 0.2747 - val_accuracy: 0.9323
Epoch 7/20
427/427 [==============================] - 95s 222ms/step - loss: 0.2234 - accuracy: 0.9384 - val_loss: 0.4504 - val_ac

In [22]:
test_loss, test_accuracy = model.evaluate(test_rescale_ds, verbose=0)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.11340105533599854
Test Accuracy: 0.9716295599937439


In [ ]:
from pathlib import Path
from tensorflow.keras.models import load_model

# Define the path where you want to save the model
model_path = Path('./models/Alternate_Lenet5.h5')

# Check if the file already exists
if model_path.exists():
    # If the file exists, remove it
    model_path.unlink()

# Save the model
model.save(model_path)

In [23]:
model.save('./models/Alternate_Lenet5.keras')

## For testing purposes

In [26]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

model = load_model('./models/Alternate_Lenet5.keras')

# Read the image file from the request
img_file = './data_all_modified-cnn-split/test/damage/-95.588835_29.769012.jpeg'

# Open the image using PIL
img = Image.open(img_file)
        
# Resize and preprocess the image
img = img.resize((128, 128))
img_array = np.array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Perform inference using the pre-trained model
prediction = model.predict(img_array)

# Extract the probability of being damaged
probability_damaged = prediction[0][0]

# Determine the class based on the threshold
threshold = 0.5
if probability_damaged >= threshold:
    prediction_label = 'No Damage'
else:
    prediction_label = 'Damage'

# Convert the prediction result into a JSON format
result = {
    'probability_no_damage': float(probability_damaged),
    'prediction': prediction_label
}

result


1/1 [==============================] - 0s 152ms/step


{'probability_no_damage': 0.00027020028210245073, 'prediction': 'Damage'}

In [29]:
train_ds.class_names

['damage', 'no_damage']

In [27]:
prediction

array([[0.0002702]], dtype=float32)